In [46]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [47]:
#Path para salvar los datos
path="H:/Mi unidad/pgd/consulta/plaza_publica"

In [ ]:
# === Configurar Chrome Headless ===
chrome_options = Options()
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")

# Crear driver
driver = webdriver.Chrome(options=chrome_options)


In [ ]:
todos=[]
for i in range(1, 23):

    url = f"https://plazapublica.cdmx.gob.mx/processes/consultaPGD2025/f/16/proposals/{i}"
    print(f"==== Descargando página {i} ====")

    try:
        driver.get(url)

        # Esperar a que aparezca el contenedor de comentarios
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".comments"))
            )
        except:

            time.sleep(5)

        # Sacar HTML
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ========= EXTRAER LÍNEA =========
        linea_tag = soup.find("h1", class_="h2 decorator")
        linea = linea_tag.get_text(strip=True).replace('"', "") if linea_tag else None

        # ========= EXTRAER COMENTARIOS =========
        comentarios_html = soup.find_all("div", class_="comment")

        print(f"Comentarios encontrados: {len(comentarios_html)}")

        for c in comentarios_html:
            # Autor
            autor_tag = c.find("span", class_="author")
            autor = autor_tag.get_text(strip=True) if autor_tag else None

            # Fechas
            time_tag = c.find("time")
            fecha_visible = time_tag.get_text(strip=True) if time_tag else None
            fecha_iso = time_tag["datetime"] if time_tag and "datetime" in time_tag.attrs else None

            # Texto comentario
            texto_tag = c.find("div", class_="comment__content")
            texto = texto_tag.get_text(strip=True) if texto_tag else None

            # Agregar a lista general
            todos.append({
                "linea": linea,
                "autor": autor,
                "fecha_visible": fecha_visible,
                "fecha_iso": fecha_iso,
                "texto": texto
            })

    except Exception as e:
        print(f"Error en página {i}: {e}")
        continue

# Cerrar driver
driver.quit()

==== Descargando página 1 ====
Comentarios encontrados: 0
==== Descargando página 2 ====
Comentarios encontrados: 0
==== Descargando página 3 ====
Comentarios encontrados: 0
==== Descargando página 4 ====
Comentarios encontrados: 0
==== Descargando página 5 ====
Comentarios encontrados: 0
==== Descargando página 6 ====
Comentarios encontrados: 0
==== Descargando página 7 ====
Comentarios encontrados: 1
==== Descargando página 8 ====
Comentarios encontrados: 0
==== Descargando página 9 ====
Comentarios encontrados: 0
==== Descargando página 10 ====
Comentarios encontrados: 0
==== Descargando página 11 ====
Comentarios encontrados: 0
==== Descargando página 12 ====
Comentarios encontrados: 0
==== Descargando página 13 ====
Comentarios encontrados: 0
==== Descargando página 14 ====
Comentarios encontrados: 0
==== Descargando página 15 ====
Comentarios encontrados: 0
==== Descargando página 16 ====
Comentarios encontrados: 0
==== Descargando página 17 ====
Comentarios encontrados: 3
==== D

In [ ]:
# Pasar a df
coment_plaza_publica = pd.DataFrame(todos)
coment_plaza_publica

,linea,autor,fecha_visible,fecha_iso,texto
0,07. De la diversidad sexo - genérica,Miguel Licea,17/11/2025 10:03,2025-11-17T10:03:22-06:00,Me parece una buena propuesta para la sociedad...
1,"17. De movilidad sustentable, articulada y acc...",Roberto González,12/11/2025 14:05,2025-11-12T14:05:51-06:00,Me gustaría que el gobierno implementara una A...
2,"17. De movilidad sustentable, articulada y acc...",Francisco Fontano Patán,17/11/2025 13:39,2025-11-17T13:39:49-06:00,Veo que la propuesta tiene muchos buenos deseo...
3,"17. De movilidad sustentable, articulada y acc...",Francisco Fontano Patán,17/11/2025 13:46,2025-11-17T13:46:24-06:00,Una cosa básica es comprometerse con un plan d...
4,20. Territorio de paz y seguridad ciudadana,Roberto González,12/11/2025 14:17,2025-11-12T14:17:54-06:00,Me gustaría que la policía y guardia nacional ...


In [51]:
###Salvar a csv
coment_plaza_publica.to_csv(path+"/comentarios_plaza_publica.csv", index=False, encoding="utf-8-sig")